In [11]:
from abc import ABC, abstractmethod
from enum import Enum

In [36]:
class Suit(Enum):
    DIAMOND=0
    SPADE=1
    HEART=2
    CLUBS=3

In [60]:
class Card(ABC):

    def __init__(self, value, suit):
        self.value = value
        self.suit = suit
        self.is_available = True

    @property
    @abstractmethod
    def value(self):
        pass

    @value.setter
    @abstractmethod
    def value(self, other_value):
        pass

class BlackJackCard(Card):

    def is_ace(self):
        return self._value == 1

    def is_face_card(self):
        return 11 <= self._value <= 13

    @property
    def value(self):
        if self.is_ace():
            return 1
        elif self.is_face_card():
            return 10
        else:
            return self._value

    @value.setter
    def value(self, new_value):
        if 1 <= new_value <= 13:
            self._value = new_value
        else:
            raise ValueError('Invalid card value: {}'.format(new_value))

    def print_value(self):
        print(f"{self._value}")

    

In [61]:
class Hand:

    def __init__(self, cards):
        self.cards = cards

    def add_card(self, card):
        self.cards.append(card)

    def score(self):
        total_value = 0
        for card in self.cards:
            total_value += card.value
        return total_value

class BlackJackHand(Hand):

    BLACKJACK = 21

    def score(self):
        min_over = float("inf")
        max_under = float("-inf")
        for score in self.possible_scores():
            if self.BLACKJACK < score < min_over:
                min_over = score
            elif max_under < score <= self.BLACKJACK:
                max_under = score
        return max_under if max_under != float("-inf") else min_over


    def possible_scores(self):
        scores = [0]
        for card in self.cards:
            new_scores = []
            for score in scores:
                if card.is_ace():
                    new_scores.append(score + 1)
                    new_scores.append(score + 11)
                else:
                    new_scores.append(score + card.value)
            scores = new_scores
        return scores


class Deck:

    def __init__(self, cards):
        self.cards = cards
        self.deal_index = 0

    def remaining_cards(self):
        return len(self.cards) - deal_index

    def deal_card(self):
        try:
            card = self.cards[self.deal_index]
            card.is_available = False
            self.deal_index += 1
            return card
        except IndexError:
            return None

    def shuffle(self):
        pass

In [67]:
import unittest

class TestBlackJackHand(unittest.TestCase):

    def setUp(self):
        # Create some cards
        self.card1 = BlackJackCard(1, Suit.HEART)   # Ace
        self.card2 = BlackJackCard(10, Suit.SPADE) # Ten
        self.card3 = BlackJackCard(5, Suit.CLUBS)  # Five
        self.card4 = BlackJackCard(12, Suit.DIAMOND) # Queen (face card)

    def test_blackjack(self):
        # Test a blackjack hand (Ace + Ten)
        hand = BlackJackHand([self.card1, self.card2])
        self.assertEqual(hand.score(), 21)

    def test_multiple_aces(self):
        # Test a hand with multiple Aces (Ace + Ace + Five)
        hand = BlackJackHand([self.card1, self.card1, self.card3])
        self.assertEqual(hand.score(), 17) # Should count one Ace as 11 and one as 1

    def test_face_card_and_ace(self):
        # Test a hand with a face card and an Ace (Ace + Queen)
        hand = BlackJackHand([self.card1, self.card4])
        self.assertEqual(hand.score(), 21)

    def test_bust_hand(self):
        # Test a bust hand (Ten + Queen + Five)
        hand = BlackJackHand([self.card2, self.card4, self.card3])
        self.assertEqual(hand.score(), 25)

    def test_regular_hand(self):
        # Test a regular hand (Five + Ten)
        hand = BlackJackHand([self.card3, self.card2])
        self.assertEqual(hand.score(), 15)

    def test_aces_varied_values(self):
        # Test hand with varied ace values (Ace + Ace + Nine)
        card5 = BlackJackCard(1, Suit.SPADE)
        card6 = BlackJackCard(9, Suit.HEART) # Nine
        hand = BlackJackHand([self.card1, card5, card6])
        self.assertEqual(hand.score(), 21) # Should count one Ace as 11 and one as 1

unittest.main(argv=[''], exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 0.003s

OK
